In [1]:
#Imports
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive

In [2]:
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive')
print("Change successful.")

Mounted at /content/drive
Change successful.


#### 1. Download the Dataset

Google Drive Link: https://drive.google.com/drive/folders/1naE0TQNJUnG0BBXPCWBkT-hNkkN2Hbjl

### 2. Image Preprocessing

#### 2.1 Import the ImageDataGenerator Library

In [3]:
from keras.preprocessing.image import ImageDataGenerator

#### 2.2 Configure ImageDataGenerator Class

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

#### 2.3 Apply ImageDataGenerator functionality to Trainset and Testset

In [5]:
x_train = train_datagen.flow_from_directory('./Dataset/train/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('./Dataset/test/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


### 3. Model Building

#### 3.1 Importing the Model Building Libraries

In [6]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

#### 3.2 Initializing the Model

In [7]:
classifier = Sequential()

#### 3.3 Adding CNN Layers

In [8]:
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1) ,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

#### 3.4 Adding Dense Layers

In [9]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax'))

In [10]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

#### 3.5 Configure the Learning Process

In [11]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### 3.6 Train the Model

In [12]:
classifier.fit_generator(
    generator=x_train, steps_per_epoch=len(x_train),
    epochs=20, validation_data=x_test, validation_steps=len(x_test)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/20
119/119 [==============================] - 337s 3s/step - loss: 1.5719 - accuracy: 0.3519 - val_loss: 1.0620 - val_accuracy: 0.5333
Epoch 2/20
119/119 [==============================] - 2s 15ms/step - loss: 0.8070 - accuracy: 0.6919 - val_loss: 0.6907 - val_accuracy: 0.7000
Epoch 3/20
119/119 [==============================] - 2s 15ms/step - loss: 0.4849 - accuracy: 0.8283 - val_loss: 0.3657 - val_accuracy: 0.9000
Epoch 4/20
119/119 [==============================] - 2s 15ms/step - loss: 0.3788 - accuracy: 0.8687 - val_loss: 0.3715 - val_accuracy: 0.9000
Epoch 5/20
119/119 [==============================] - 2s 15ms/step - loss: 0.2111 - accuracy: 0.9360 - val_loss: 0.3378 - val_accuracy: 0.9333
Epoch 6/20
119/119 [==============================] - 2s 15ms/step - loss: 0.2193 - accuracy: 0.9226 - val_loss: 0.2842 - val_accuracy: 0.9333
Epoch 7/20
119/119 [==============================] - 2s 15ms/step - loss: 0.1718 - accuracy: 0.9394 - val_loss: 0.2279 - val_accuracy: 0.9000

#### 3.7 Save the Model

In [13]:
classifier.save('gesture.h5')

In [14]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as f:
    f.write(model_json)

#### 3.8 Test the Model

In [15]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model('gesture.h5')

In [16]:
from tensorflow.keras.utils import load_img, img_to_array
img = load_img("./Dataset/test/0/0.jpg", grayscale=True, target_size=(64, 64))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)

1/1 [==============================] - 0s 123ms/step


/usr/local/lib/python3.7/dist-packages/keras/utils/image_utils.py:382: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [17]:
pred

array([[1., 0., 0., 0., 0., 0.]], dtype=float32)

In [18]:
pred[0]

array([1., 0., 0., 0., 0., 0.], dtype=float32)